# !!! Все выводы сделаны при прогоне скрипта. При запуске с нуля цифры могут отличаться

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, classification_report

In [4]:
import warnings
warnings.filterwarnings('ignore')

## Загрузка и обработка данных

In [5]:
data = pd.read_csv('train.csv')

In [6]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
data.shape

(891, 12)

In [8]:
data.nunique()

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

In [9]:
data.Cabin.unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

In [10]:
#исключаем PassengerId, Name, Ticket - они уникальны (или практически уникальны)
data = data.drop(columns=['PassengerId', 'Name', 'Ticket'])
#заменяем Cabin на признак, обозначающий, пропущено значение или нет
data['Cabin'] = data['Cabin'].isna().astype(float)

In [11]:
#проверка - есть ли пропущенные значения и в каких столбцах
data.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin         0
Embarked      2
dtype: int64

In [12]:
#заменяем пропуски в Age медианой
data['Age'] = data['Age'].fillna(data['Age'].median())
#удаляем строки с пропуском в Embarked - их мало
data = data.dropna()

In [13]:
#делим выборку на train и test
X = data.drop(columns='Survived')
y = data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y, )

In [14]:
#проверка сбалансированности классов
y.mean()

0.38245219347581555

In [15]:
# преобразование категориальных переменных OneHotEncoder
cat_features = ['Pclass', 'Sex', 'Embarked']
ohe = OneHotEncoder(drop='first', sparse=False)
X_train_cat = pd.DataFrame(ohe.fit_transform(X_train[cat_features]), columns=ohe.get_feature_names_out())
X_train = pd.concat([X_train.drop(columns=cat_features).reset_index(drop=True), X_train_cat], axis=1)
X_test_cat = pd.DataFrame(ohe.transform(X_test[cat_features]), columns=ohe.get_feature_names_out())
X_test = pd.concat([X_test.drop(columns=cat_features).reset_index(drop=True), X_test_cat], axis=1)

## Обучение моделей

In [16]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
pred_train = logreg.predict(X_train)
pred_test = logreg.predict(X_test)
proba_train = logreg.predict_proba(X_train)[:, 1]
proba_test = logreg.predict_proba(X_test)[:, 1]
print('Train: ')
print(f'ROC_AUC {roc_auc_score(y_train, proba_train):0.2}')
print(classification_report(y_train, pred_train))
print('Test: ')
print(f'ROC_AUC {roc_auc_score(y_test, proba_test):0.2}')
print(classification_report(y_test, pred_test))

Train: 
ROC_AUC 0.85
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       411
           1       0.77      0.71      0.73       255

    accuracy                           0.80       666
   macro avg       0.80      0.79      0.79       666
weighted avg       0.80      0.80      0.80       666

Test: 
ROC_AUC 0.87
              precision    recall  f1-score   support

           0       0.84      0.89      0.86       138
           1       0.80      0.72      0.76        85

    accuracy                           0.83       223
   macro avg       0.82      0.80      0.81       223
weighted avg       0.82      0.83      0.82       223



In [17]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
pred_train = knn.predict(X_train)
pred_test = knn.predict(X_test)
proba_train = knn.predict_proba(X_train)[:, 1]
proba_test = knn.predict_proba(X_test)[:, 1]
print('Train: ')
print(f'ROC_AUC {roc_auc_score(y_train, proba_train):0.2}')
print(classification_report(y_train, pred_train))
print('Test: ')
print(f'ROC_AUC {roc_auc_score(y_test, proba_test):0.2}')
print(classification_report(y_test, pred_test))

Train: 
ROC_AUC 0.87
              precision    recall  f1-score   support

           0       0.82      0.86      0.84       411
           1       0.76      0.70      0.73       255

    accuracy                           0.80       666
   macro avg       0.79      0.78      0.78       666
weighted avg       0.80      0.80      0.80       666

Test: 
ROC_AUC 0.77
              precision    recall  f1-score   support

           0       0.75      0.79      0.77       138
           1       0.63      0.58      0.60        85

    accuracy                           0.71       223
   macro avg       0.69      0.68      0.69       223
weighted avg       0.70      0.71      0.71       223



In [30]:
print('''
У обеих моделей значения метрик достаточно высокие.
У модели KNN на тренировочной выборке ROC_AUC несколько выше (при не очень сильном различии прочих метрик), а на 
тестовой - ниже, но все-таки я считаю, что KNN может несколько лучше ранжировать вероятности
''')


У обеих моделей значения метрик достаточно высокие.
У модели KNN на тренировочной выборке ROC_AUC несколько выше (при не очень сильном различии прочих метрик), а на 
тестовой - ниже, но все-таки я считаю, что KNN может несколько лучше ранжировать вероятности



## Подбор гиперпараметров

In [19]:
logreg_params = dict(penalty=[None, 'l1', 'l2', 'elasticnet'], C=np.logspace(-10, 2, 500), class_weight=[None, 'balanced'])
logreg = LogisticRegression()
cv_logreg = RandomizedSearchCV(logreg, logreg_params, n_iter=100, n_jobs=-1)
cv_logreg.fit(X_train, y_train)

RandomizedSearchCV(estimator=LogisticRegression(), n_iter=100, n_jobs=-1,
                   param_distributions={'C': array([1.00000000e-10, 1.05693455e-10, 1.11711065e-10, 1.18071285e-10,
       1.24793621e-10, 1.31898690e-10, 1.39408283e-10, 1.47345431e-10,
       1.55734477e-10, 1.64601150e-10, 1.73972643e-10, 1.83877698e-10,
       1.94346693e-10, 2.05411735e-10, 2.17106760e-10, 2.29467637e-10,
       2.42532...
       3.49209598e+01, 3.69091691e+01, 3.90105762e+01, 4.12316259e+01,
       4.35791301e+01, 4.60602884e+01, 4.86827104e+01, 5.14544388e+01,
       5.43839743e+01, 5.74803016e+01, 6.07529169e+01, 6.42118571e+01,
       6.78677305e+01, 7.17317494e+01, 7.58157646e+01, 8.01323013e+01,
       8.46945981e+01, 8.95166472e+01, 9.46132376e+01, 1.00000000e+02]),
                                        'class_weight': [None, 'balanced'],
                                        'penalty': [None, 'l1', 'l2',
                                                    'elasticnet']})

In [20]:
cv_logreg.best_params_

{'penalty': 'l2', 'class_weight': None, 'C': 10.328281259410275}

In [21]:
print('''
Подбор оптимальных параметров для логистической регрессии - подбираются параметры:
    penalty - параметр типа регуляризации,
    С - параметр силы регуляризации,
    class_weight - параметр, отвечающий за то, будут ли сбалансированы классы при обучении.
    
Результат подбора параметров: наилучшая регуляризация l2, вес классов учитывать не следует (классы и так достаточно
сбалансированы, см. выше), параметр регуляризации С достаточно мал => необходима достаточно сильная регуляризация 
(модель достаточно сильно переобучается)
''')


Подбор оптимальных параметров для логистической регрессии - подбираются параметры:
    penalty - параметр типа регуляризации,
    С - параметр силы регуляризации,
    class_weight - параметр, отвечающий за то, будут ли сбалансированы классы при обучении.
    
Результат подбора параметров: наилучшая регуляризация l2, вес классов учитывать не следует (классы и так достаточно
сбалансированы, см. выше), параметр регуляризации С достаточно мал => необходима достаточно сильная регуляризация 
(модель достаточно сильно переобучается)



In [22]:
pred_train = cv_logreg.predict(X_train)
pred_test = cv_logreg.predict(X_test)
proba_train = cv_logreg.predict_proba(X_train)[:, 1]
proba_test = cv_logreg.predict_proba(X_test)[:, 1]
print('Train: ')
print(f'ROC_AUC {roc_auc_score(y_train, proba_train):0.2}')
print(classification_report(y_train, pred_train))
print('Test: ')
print(f'ROC_AUC {roc_auc_score(y_test, proba_test):0.2}')
print(classification_report(y_test, pred_test))

Train: 
ROC_AUC 0.85
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       411
           1       0.77      0.71      0.73       255

    accuracy                           0.80       666
   macro avg       0.80      0.79      0.79       666
weighted avg       0.80      0.80      0.80       666

Test: 
ROC_AUC 0.87
              precision    recall  f1-score   support

           0       0.84      0.88      0.86       138
           1       0.79      0.72      0.75        85

    accuracy                           0.82       223
   macro avg       0.81      0.80      0.81       223
weighted avg       0.82      0.82      0.82       223



In [23]:
print('Переобучениие модели слегка снизилось')

Переобучениие модели слегка снизилось


In [24]:
knn_params = dict(n_neighbors=np.arange(1, 100), p=[1, 2], weights=['uniform', 'distance'])
knn = knn = KNeighborsClassifier()
cv_knn = RandomizedSearchCV(knn, knn_params, n_iter=100)
cv_knn.fit(X_train, y_train)

RandomizedSearchCV(estimator=KNeighborsClassifier(), n_iter=100,
                   param_distributions={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
                                        'p': [1, 2],
                                        'weights': ['uniform', 'distance']})

In [25]:
cv_knn.best_params_

{'weights': 'distance', 'p': 1, 'n_neighbors': 55}

In [26]:
print('''
Подбор оптимальных параметров для KNN - подбираются параметры:
    weights - будет ли зависеть ответ от близости объекта или нет,
    р - метрика, по которой считается расстояние,
    n_neighbors - параметр, отвечающий за то, будут ли сбалансированы классы при обучении.
    
Результат подбора параметров: метрика расстояния l1, количество соседей 55 (учитывается большое число соседей),
ответ зависит от того, насколько далеко расположены соседи
''')


Подбор оптимальных параметров для KNN - подбираются параметры:
    weights - будет ли зависеть ответ от близости объекта или нет,
    р - метрика, по которой считается расстояние,
    n_neighbors - параметр, отвечающий за то, будут ли сбалансированы классы при обучении.
    
Результат подбора параметров: метрика расстояния l1, количество соседей 54 (учитывается большое число соседей),
ответ зависит от того, насколько далеко расположены соседи



In [27]:
pred_train = cv_knn.predict(X_train)
pred_test = cv_knn.predict(X_test)
proba_train = cv_knn.predict_proba(X_train)[:, 1]
proba_test = cv_knn.predict_proba(X_test)[:, 1]
print('Train: ')
print(f'ROC_AUC {roc_auc_score(y_train, proba_train):0.2}')
print(classification_report(y_train, pred_train))
print('Test: ')
print(f'ROC_AUC {roc_auc_score(y_test, proba_test):0.2}')
print(classification_report(y_test, pred_test))

Train: 
ROC_AUC 1.0
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       411
           1       1.00      0.98      0.99       255

    accuracy                           0.99       666
   macro avg       0.99      0.99      0.99       666
weighted avg       0.99      0.99      0.99       666

Test: 
ROC_AUC 0.77
              precision    recall  f1-score   support

           0       0.77      0.87      0.82       138
           1       0.73      0.58      0.64        85

    accuracy                           0.76       223
   macro avg       0.75      0.72      0.73       223
weighted avg       0.75      0.76      0.75       223



In [31]:
print('''
С новыми параметрами модель переобучилась - на тренировочной выборке результат практически "идеальный", 
на тестовой результат не изменился.

Вывод - следует использовать другой набор параметров.
''')


С новыми параметрами модель переобучилась - на тренировочной выборке результат практически "идеальный", 
на тестовой результат не изменился.

Вывод - следует использовать другой набор параметров.

